## Cleaning Data For Machine Learning

This tutorial is based upon the textbook:

Walker, M. (2022). Data Cleaning and Exploration with Machine Learning. Pakt Publishing Ltd..

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn.impute import KNNImputer

pd.options.display.float_format = '{:,.0f}'.format


## Missing values - more of a problem than you think

Missing values are one of the trickiest issues in data science because they can bias results in ways that aren’t always obvious. The biggest concern is when values are not missing at random—for example, if people with lower education are less likely to report their earnings, the missingness itself is correlated with the variable of interest. In such cases, simply ignoring or deleting missing values could distort your analysis.

Once missing values are identified, the challenge becomes deciding how to handle them. There is no single correct approach—different strategies make sense depending on the context.
* Deletion (removing rows or columns with too many missing values) can be acceptable if the dataset is large and the missingness is small or random.
* Simple imputation, such as filling with the mean, median, or mode, is quick but risks underestimating variability.
* Conditional imputation, such as using group-specific means (e.g., mean within a category) or time-series methods like forward/backward filling, often provides more meaningful estimates.
* More advanced methods, like regression-based imputation or K-nearest neighbors (KNN) imputation, exploit multivariate relationships in the data to generate plausible replacements.

Several important considerations are often overlooked. First, missing data handling should always be done after the train–test split to avoid leakage—statistics used for imputation (like means or regression models) must come from the training data only. Second, no single method works best in all cases: deletion, simple imputation, and advanced imputation are all tools that should be chosen depending on sample size, the amount of missingness, and the mechanism causing missingness (random, systematic, or related to unobserved factors).

Finally, it’s crucial to remember that imputation introduces uncertainty—good practice is to assess the sensitivity of results by comparing multiple imputation strategies.

In [2]:
nls97 = pd.read_csv("nls97b.csv")
nls97.set_index("personid", inplace=True)
covidtotals = pd.read_csv("covidtotals.csv")
covidtotals.set_index("iso_code", inplace=True)

covidtotals.shape

(221, 16)

# Finding
We will count the number of missing values for columns that we may use as
features. We can use the isnull method to test whether each feature value is
missing. It will return True if the value is missing and False if not. Then, we
can use sum to count the number of True values since sum will treat each True
value as 1 and each False value as 0. We use axis=0 to sum over the rows for
each column:

In [3]:
demovars = ['population_density','aged_65_older',
   'gdp_per_capita','life_expectancy','diabetes_prevalence']
covidtotals[demovars].isnull().sum(axis=0)


population_density     15
aged_65_older          33
gdp_per_capita         28
life_expectancy         4
diabetes_prevalence    21
dtype: int64

If we want the number of missing values for each row, we can specify axis=1
when summing. The following code creates a Series, demovarsmisscnt, with
the number of missing values for the demographic features for each country. 181
countries have values for all of the features, 11 are missing values for four of the five
features, and three are missing values for all of the features:

In [4]:
demovarsmisscnt = covidtotals[demovars].isnull().sum(axis=1)
demovarsmisscnt.value_counts().sort_index()



0    181
1     15
2      6
3      5
4     11
5      3
Name: count, dtype: int64

Let's take a look at a few of the countries with four or more missing values. There is
very little demographic data available for these countries:

In [5]:
covidtotals.loc[demovarsmisscnt>=4, ['location'] + demovars]


,location,population_density,aged_65_older,gdp_per_capita,life_expectancy,diabetes_prevalence
iso_code,,,,,,
AIA,Anguilla,NaN,NaN,NaN,82,NaN
BES,Bonaire Sint Eustatius and Saba,NaN,NaN,NaN,78,NaN
COK,Cook Islands,NaN,NaN,NaN,76,NaN
FLK,Falkland Islands,NaN,NaN,NaN,81,NaN
GGY,Guernsey,NaN,NaN,NaN,NaN,NaN
JEY,Jersey,NaN,NaN,NaN,NaN,NaN
MSR,Montserrat,NaN,NaN,NaN,74,NaN
NIU,Niue,NaN,NaN,NaN,74,NaN
PCN,Pitcairn,NaN,NaN,NaN,NaN,NaN


Let's also check missing values for total cases and deaths. 29 countries have missing
values for cases per million in population, and 36 have missing deaths per million

In [6]:
# check the cumulative columns for missing
totvars = ['location','total_cases_mill','total_deaths_mill']
covidtotals[totvars].isnull().sum(axis=0)
totvarsmisscnt = covidtotals[totvars].isnull().sum(axis=1)
totvarsmisscnt.value_counts().sort_index()



0    185
1      7
2     29
Name: count, dtype: int64

### Coded missing values

ometimes, we have logical missing values that we need to transform into actual
missing values. This happens when the dataset designers use valid values as codes
for missing values. These are often values such as 9, 99, or 999, based on the
allowable number of digits for the variable. Or it might be a more complicated
coding scheme where there are codes for different reasons for there being missings.
For example, in the NLS dataset, the codes reveal why the respondent did not
provide an answer for a question: -3 is an invalid skip, -4 is a valid skip, and -5 is
a non-interview.

In [7]:
# set logical missings to actual missings
nlsparents = nls97.iloc[:,-4:]
nlsparents.shape
nlsparents.loc[nlsparents.motherhighgrade.between(-5,-1), 'motherhighgrade'].value_counts()


motherhighgrade
-3    523
-4    165
Name: count, dtype: int64

For our analysis, the reason why there is a non-response is not important. Let's just
count the number of non-responses for each of the features, regardless of the reason
for the non-response:

In [8]:
nlsparents.replace(list(range(-5,0)), np.nan, inplace=True)
nlsparents.isnull().sum()

motherage           608
parentincome       2396
fatherhighgrade    1856
motherhighgrade     688
dtype: int64

## Cleaning missing values
In this section, we'll go over some of the most straightforward approaches for handling
missing values. This includes dropping observations where there are missing values;
assigning a sample-wide summary statistic, such as the mean, to the missing values; and
assigning values based on the mean value for an appropriate subset of the data

In [9]:
# set up school record and demographic data frames from the NLS data
schoolrecordlist = ['satverbal','satmath','gpaoverall','gpaenglish',
  'gpamath','gpascience','highestdegree','highestgradecompleted']

schoolrecord = nls97[schoolrecordlist]
print(f"Shape: {schoolrecord.shape}")
# check the school record data for missings
print(schoolrecord.isnull().sum(axis=0))




Shape: (8984, 8)
satverbal                7578
satmath                  7577
gpaoverall               2980
gpaenglish               3186
gpamath                  3218
gpascience               3300
highestdegree              31
highestgradecompleted    2321
dtype: int64


We can create a Series, misscnt, that specifies the number of missing features
for each observation with misscnt = schoolrecord.isnull().
sum(axis=1). 946 observations have seven missing values for the educational
data, while 11 are missing values for all eight features:

In [10]:

misscnt = schoolrecord.isnull().sum(axis=1)
print(misscnt.value_counts().sort_index())


0    1087
1     312
2    3210
3    1102
4     176
5     101
6    2039
7     946
8      11
Name: count, dtype: int64


Let's drop observations that have missing values for seven or more features out of
eight. We can accomplish this by setting the thresh parameter of dropna to 2.
This will drop observations that have fewer than two non-missing values; that is, 0
or 1 non-missing values. We get the expected number of observations after using
dropna; that is, 8,984 - 946 - 11 = 8,027

In [11]:
schoolrecord = schoolrecord.dropna(thresh=2)
schoolrecord.shape

(8027, 8)

# Simple replacement

The most straightforward approach is to assign the overall mean for gpaoverall
to the missing values. The following code uses the pandas Series fillna method
to assign all missing values of gpaoverall to the mean value of the Series. The
first argument to fillna is the value you want for all missing values – in this case,
schoolrecord.gpaoverall.mean().
Note that we need to remember to set
the inplace parameter to True to overwrite the existing values:

In [12]:
# assign mean values to missings
schoolrecord.gpaoverall.agg(['mean','std','count'])


mean      282
std        62
count   6,004
Name: gpaoverall, dtype: float64

In [12]:
schoolrecord["gpaoverall"] = schoolrecord["gpaoverall"].fillna(
    schoolrecord["gpaoverall"].mean()
)

print(schoolrecord.gpaoverall.isnull().sum())
print(schoolrecord.gpaoverall.agg(['mean','std','count']))



0
mean      282
std        53
count   8,027
Name: gpaoverall, dtype: float64


The mean has not changed. However, there is a substantial reduction in the standard
deviation, from 61.6 to 53.3. This is one of the disadvantages of using the dataset's
mean for all missing values.

### Forward fill
The NLS data also has a fair number of missing values for wageincome. 
Rather than assigning the mean value of wageincome to the missings, we could
use another common technique for imputing values: we could assign the nearest
non-missing value from a preceding observation. The ffill option of fillna
will do this for us:

In [14]:
# use forward fill
wageincome = nls97.wageincome.copy(deep=True)
print(wageincome.isnull().sum())
print(wageincome.agg(['mean','std','count']))


3893
mean    49,477
std     40,678
count    5,091
Name: wageincome, dtype: float64


In [15]:
wageincome.fillna(method='ffill', inplace=True)
print(wageincome.isnull().sum())
print(wageincome.agg(['mean','std','count']))



0
mean    49,549
std     40,014
count    8,984
Name: wageincome, dtype: float64


C:\Users\Ian\AppData\Local\Temp\ipykernel_28340\2023583247.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  wageincome.fillna(method='ffill', inplace=True)


We could have done a backward fill instead by setting the method parameter of
fillna to bfill. This sets missing values to the nearest following value. This
produces the following output:

In [16]:
wageincome = nls97.wageincome.copy(deep=True)
wageincome.fillna(method='bfill', inplace=True)
print(wageincome.agg(['mean','std','count']))



mean    49,419
std     41,112
count    8,984
Name: wageincome, dtype: float64


C:\Users\Ian\AppData\Local\Temp\ipykernel_28340\4197383777.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  wageincome.fillna(method='bfill', inplace=True)


If missing values are randomly distributed, then forward or backward filling has one
advantage over using the mean: it is more likely to approximate the distribution of
the non-missing values for the feature. Notice that the standard deviation did not
drop substantially.
There are times when it makes sense to base our imputation of values on the mean
or median value for similar observations; say, those that have the same value for a
related feature. If we are imputing values for feature X1, and X1 is correlated with
X2, we can use the relationship between X1 and X2 to impute a value for X1 that
may make more sense than the dataset's mean. This is pretty straightforward when
X2 is categorical. In this case, we can impute the mean value of X1 for the associated
value of X2.

### Using groups

In the NLS DataFrame, weeks worked in 2017 correlates with the highest degree
earned. The following code shows how the mean value of weeks worked changes
with degree attainment. The mean for weeks worked is 39, but it is much lower for
those without a degree (28.72) and much higher for those with a professional degree
(47.20). In this case, it may be a better choice to assign 28.72 to the missing values
for weeks worked for individuals who have not attained a degree, rather than 39:

In [17]:
# fill missings with the average by group
print(nls97.weeksworked17.mean())
print(nls97.groupby(['highestdegree'])['weeksworked17'].mean())


39.01664167916042
highestdegree
0. None           29
1. GED            35
2. High School    38
3. Associates     40
4. Bachelors      44
5. Masters        45
6. PhD            44
7. Professional   47
Name: weeksworked17, dtype: float64


The following code assigns the mean value of weeks worked across observations
with the same degree attainment level, for those observations missing weeks
worked. We do this by using groupby to create a groupby DataFrame,
groupby(['highestdegree'])['weeksworked17']. Then, we use
fillna within apply to fill those missing values with the mean for the highest
degree group. Notice that we make sure to only do this imputation for observations
where the highest degree is not missing, ~nls97.highestdegree.isnull().
We will still have missing values for observations that are missing both the highest
degree and weeks worked:

In [18]:
means = nls97.groupby("highestdegree")["weeksworked17"].mean()
nls97["weeksworked17imp"] = nls97["weeksworked17"].fillna(
    nls97["highestdegree"].map(means)
)
nls97[['weeksworked17imp','weeksworked17',
'highestdegree']].head(10)

,weeksworked17imp,weeksworked17,highestdegree
personid,,,
100061,48,48,2. High School
100139,52,52,2. High School
100284,0,0,0. None
100292,44,NaN,4. Bachelors
100583,52,52,2. High School
100833,47,47,2. High School
100931,52,52,3. Associates
101089,52,52,2. High School
101122,38,NaN,2. High School


These imputation strategies – removing observations with missing values, assigning a
dataset's mean or median, using forward or backward filling, or using a group mean for
a correlated feature – are fine for many predictive analytics projects. They work best
when the missing values are not correlated with the target. When that is true, imputing
values allows us to retain the other information from those observations without biasing
our estimates.

## Regression based imputation

Regression imputation is a method for handling missing values by predicting them with a regression model built from other correlated features. Instead of filling missing values with an overall mean or a group mean, regression imputation leverages multiple predictors to produce more informed estimates. The simplest version, **deterministic regression imputation**, replaces missing values with predictions that lie exactly on the regression line. This ensures consistency with the model but artificially reduces variability in the data, since the imputed values are "too perfect." To address this, **stochastic regression imputation** adds a random error term (drawn from the model’s residuals) to the predicted values, preserving some of the natural variation.

Regression imputation is most useful when the variable with missing data is strongly related to several other features. It can yield better imputations than single-statistic methods (like mean or median imputation) because it respects relationships between variables. However, students should be aware of several caveats:

* **Variance reduction**: Deterministic regression imputation can underestimate variability, making the imputed variable look less dispersed than it truly is. This can bias downstream analyses.
* **Stochastic correction**: Adding a residual component helps maintain variance, but it introduces randomness, which can complicate reproducibility unless carefully managed (e.g., with fixed random seeds).
* **Risk of overfitting**: The regression model used for imputation must be trained **only on the training data**, otherwise information leaks from the test set.
* **Distributional mismatch**: If the regression model is misspecified (e.g., assuming linearity when the relationship is nonlinear), imputations may distort the data distribution.
* **Computational cost**: Compared to simple imputation, regression methods require extra modeling and computation.

In practice, regression imputation works well when the relationships between features are stable and well understood. For large datasets or where missingness is limited, simpler methods may be more practical. For smaller datasets or when accuracy is critical, regression or even more advanced approaches like multiple imputation or machine-learning–based imputers (e.g., random forest, KNN) may be preferred.

Regression imputation is a powerful tool because it uses information from multiple predictors, but you must be cautious about variance shrinkage, possible overfitting, and data leakage. Stochastic regression imputation often provides a better balance by preserving variability.

In [19]:
# check correlations with wageincome
nls97[['wageincome','highestdegree','weeksworked16','parentincome']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 8984 entries, 100061 to 999963
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   wageincome     5091 non-null   float64
 1   highestdegree  8953 non-null   object 
 2   weeksworked16  7068 non-null   float64
 3   parentincome   8984 non-null   int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 350.9+ KB


Let's convert the highestdegree feature into a numeric value. This will make the
analysis we'll be doing in the rest of this section easier

In [20]:
nls97['hdegnum'] = nls97.highestdegree.str[0:1].astype('float')
nls97.groupby(['highestdegree','hdegnum']).size()


highestdegree    hdegnum
0. None          0           953
1. GED           1          1146
2. High School   2          3667
3. Associates    3           737
4. Bachelors     4          1673
5. Masters       5           603
6. PhD           6            54
7. Professional  7           120
dtype: int64

In [21]:
# Replace invalid parent income values (-5 to -1) with NaN (mark them as missing)
nls97 = nls97.copy()
nls97['parentincome'] = nls97['parentincome'].mask(
    nls97['parentincome'].between(-5, -1), np.nan
)

# Compute correlations among income, education, work experience, and parent income
corr_matrix = nls97[['wageincome', 'hdegnum', 'weeksworked16', 'parentincome']].corr()




We should check whether observations with missing values for wage income are
different in some important way from those with non-missing values. The following
code shows that these observations have significantly lower degree attainment
levels, parental income, and weeks worked. This is a clear case where assigning the
overall mean would not be the best choice:

In [22]:
# Create an indicator for missing wage income: 1 if missing, 0 otherwise
nls97 = nls97.assign(
    missingwageincome = nls97['wageincome'].isna().astype(int)
)

# Compare characteristics of people with and without missing wage income
# (average and count of degree, parent income, weeks worked)
missing_profile = (
    nls97
    .groupby('missingwageincome')[['hdegnum', 'parentincome', 'weeksworked16']]
    .agg(['mean','count'])
)

print(missing_profile)


                  hdegnum       parentincome       weeksworked16      
                     mean count         mean count          mean count
missingwageincome                                                     
0                       3  5072       48,409  3803            48  5052
1                       2  3881       43,566  2785            16  2016


Let's try regression imputation instead. Let's start by cleaning up the data a little
bit more. We can replace the missing weeksworked16 and parentincome
values with their means. We should also collapse hdegnum into those attaining less
than a college degree, those with a college degree, and those with a post-graduate
degree. We can set those up as dummy variables, with 0 or 1 values when they're
False or True, respectively. This is a tried and true method for treating categorical
data in regression analysis as it allows us to estimate different y intercepts based on
group membership:

In [23]:
# Fill missing values in predictors with their mean (avoids dropping rows later)
nls97['weeksworked16'] = nls97['weeksworked16'].fillna(nls97['weeksworked16'].mean())
nls97['parentincome']  = nls97['parentincome'].fillna(nls97['parentincome'].mean())

# Create education dummy variables while preserving missingness:
# We use np.where with a mask `~nls97['hdegnum'].isna()` so that when hdegnum is NaN,
# the dummy remains NaN (rather than 0, which would incorrectly imply "not that level").

nls97['degltcol'] = (nls97['hdegnum'] <= 2).astype(int)        # less than college
nls97['degcol']   = (nls97['hdegnum'].between(3, 4)).astype(int) # college degree
nls97['degadv']   = (nls97['hdegnum'] > 4).astype(int)         # advanced degree




In [24]:
# --- Step 4. Regression Helper Function ---
def getlm(df, ycolname, xcolnames):
    """
    Fits an OLS regression with ycolname as dependent variable
    and xcolnames as predictors. Returns coefficients and model.
    """
    # Keep only y and predictors, drop missing rows
    model_df = df[[ycolname] + xcolnames].dropna()
    y = model_df[ycolname]
    X = sm.add_constant(model_df[xcolnames])  # add intercept
    
    # Fit linear regression
    lm = sm.OLS(y, X).fit()
    
    # Collect coefficients and p-values into a DataFrame
    coefficients = pd.DataFrame(
        {
            'features': ['constant'] + xcolnames,
            'params':   lm.params.values,
            'pvalues':  lm.pvalues.values
        }
    )
    return coefficients, lm

# Choose predictors and run regression
xvars = ['weeksworked16', 'parentincome', 'degcol', 'degadv']
nls97[xvars].head()



,weeksworked16,parentincome,degcol,degadv
personid,,,,
100061,48,"7,400",0,0
100139,53,"57,000",0,0
100284,47,"50,000",0,0
100292,4,"62,760",1,0
100583,53,"18,500",0,0


Now, we can use the getlm function to get the parameter estimates and the model
summary. All of the coefficients are positive and significant at the 95% level since
they have pvalues less than 0.05. As expected, wage income increases with the
number of weeks worked and with parental income. Having a college degree gives a
nearly $16K boost to earnings, compared with not having a college degree. A postgraduate degree bumps up the earnings prediction even more – almost $37K more
than for those with less than a college degree:

In [25]:
coefficients, lm = getlm(nls97, 'wageincome', xvars)

We can use this model to impute values for wage income where they are missing.
We need to add a constant for the predictions since our model included a constant.
We can convert the predictions into a DataFrame and then join it with the rest of
the NLS data. Then, we can create a new wage income feature, wageincomeimp,
that gets the predicted value when wage income is missing, and the original wage
income value otherwise. Let's also take a look at some of the predictions to see
whether they make sense:

In [26]:
# Generate predictions for wage income using the regression model
predict = lm.predict(sm.add_constant(nls97[xvars])).rename('pred')

# add to data
nls97 = nls97.assign(pred = lm.predict(sm.add_constant(nls97[xvars])))
nls97['wageincomeimp'] = nls97['wageincome'].fillna(nls97['pred'])

# Preview and summary 
pd.options.display.float_format = '{:,.0f}'.format
preview = nls97[['wageincomeimp', 'wageincome']].join(nls97[xvars]).head(10)
summary = nls97[['wageincomeimp','wageincome']].agg(['count','mean','std'])

print(preview)
print(summary)  


          wageincomeimp  wageincome  weeksworked16  parentincome  degcol  \
personid                                                                   
100061           12,500      12,500             48         7,400       0   
100139          120,000     120,000             53        57,000       0   
100284           58,000      58,000             47        50,000       0   
100292           36,547         NaN              4        62,760       1   
100583           30,000      30,000             53        18,500       0   
100833           39,000      39,000             45        37,000       0   
100931           56,000      56,000             53        60,200       1   
101089           36,000      36,000             53        32,307       0   
101122           35,151         NaN             39        46,362       0   
101132                0           0             22         2,470       0   

          degadv  
personid          
100061         0  
100139         0  
100284     

Stochastic regression imputation adds a normally distributed error to the
predictions based on the residuals from our model. We want this error to have a
mean of 0 with the same standard deviation as our residuals. We can use NumPy's
normal function for that with np.random.normal(0, lm.resid.std(),
nls97.shape[0]). The lm.resid.std() parameter gets us the standard
deviation of the residuals from our model. The final parameter value, nls97.
shape[0], indicates how many values to create; in this case, we want a value for
every row in our data.

In [27]:
# Stochastic Imputation (adds randomness) ---
# Generate random noise from a normal distribution with same variance as residuals
randomadd = np.random.normal(loc=0, scale=lm.resid.std(), size=nls97.shape[0])

# Add random noise, stochastic predictions, and stochastically imputed wages
nls97 = nls97.assign(
    randomadd = randomadd,
    stochasticpred = nls97['pred'] + randomadd,
    # If wage income is missing, use stochastic prediction instead of pure prediction
    wageincomeimpstoc = nls97['wageincome'].fillna(lambda s: nls97['stochasticpred'])
)



Regression imputation is a good way to take advantage of all the data we have to impute
values for a feature. It is often superior to the imputation methods we examined in the
previous section, particularly when missing values are not random. If we use stochastic
regression imputation, we will not artificially reduce our variance.
Before we started using machine learning for this work, this was our go-to multivariate
approach for imputation. We now have the option of using algorithms such as KNN
for this task, which has advantages over regression imputation in some cases. KNN
imputation, unlike regression imputation, does not assume a linear relationship between
features, or that those features are normally distributed. We will explore KNN imputation
in the next section.

## KNN Imputation

KNN is a popular machine learning technique because it is intuitive, easy to run, and
yields good results when there are not a large number of features and observations. For
the same reasons, it is often used to impute missing values. As its name suggests, KNN
identifies the k observations whose features are most similar to each observation. When
it's used to impute missing values, KNN uses the nearest neighbors to determine what fill
values to use.
We can use KNN imputation to do the same imputation we did in the previous section on
regression imputation

In [28]:
has_hdeg = ~nls97['hdegnum'].isna()

# Replace invalid/sentinel parent income values (-5, -4, -3, -2, -1) with NaN.
nls97['parentincome'] = nls97['parentincome'].mask(nls97['parentincome'].between(-5, -1), np.nan)

# -----------------------------------------
# Select features to impute and build matrix
# -----------------------------------------

# Columns to include in the imputation model.
# KNNImputer will impute missing values by looking at nearest neighbors in this feature space.
wagedatalist = ['wageincome', 'weeksworked16', 'parentincome', 'degltcol', 'degcol', 'degadv']

# Extract the working frame; ensure it exists and is numeric (KNNImputer requires floats).
wagedata = nls97[wagedatalist].astype('float')

# --------------------------------------------------------
# Configure KNNImputer (protect against too-large k values)
# --------------------------------------------------------

# How many rows do we have with at least one non-missing value? (A rough proxy for usable rows)
n_valid_rows = len(wagedata)
print(f'Number of rows with at least one non-missing value: {n_valid_rows}')




Number of rows with at least one non-missing value: 8984


We are now ready to use the fit_transform method of the KNN imputer to
get values for all the missing values in the passed DataFrame, wagedata. fit_
transform returns a NumPy array that contains all the non-missing values from
wagedata, plus the imputed ones. We can convert this array into a DataFrame
using the same index as wagedata. This will make it easy to join the data in the
next step.

We need to specify the value to use for the number of nearest neighbors, for k. We
use a general rule of thumb for determining k – the square root of the number of
observations divided by 2 (sqrt(N)/2). That gives us 47 for k in this case.

In [29]:
# n_neighbors must be < number of samples to work correctly; guard against edge cases.
k = min(47, max(1, n_valid_rows - 1))

# Initialize the imputer:
# - KNNImputer uses Euclidean distance on standardized feature space implicitly (not scaled),
#   so be aware that features on different scales can dominate distance.
#   (For teaching: you can standardize features first if appropriate.)
impKNN = KNNImputer(n_neighbors=k)

# Fit the imputer on the data and transform to fill in missing values.
# Returns a NumPy array with the same shape, with imputed values where NaNs were present.
newvalues = impKNN.fit_transform(wagedata)

# Name the imputed columns.
# For continuous vars we add 'imp' suffix; dummies are also imputed but we keep original names
# to easily compare behavior. If you want, you can round or re-binarize imputed dummies later.
wagedatalistimp = ['wageincomeimpknn', 'weeksworked16imp', 'parentincomeimp', 'degltcol', 'degcol', 'degadv']

# Convert the imputed array back to a DataFrame, keeping the original index alignment.
wagedataimpknn = pd.DataFrame(newvalues, columns=wagedatalistimp, index=wagedata.index)
wagedataimpknn.head(10)


,wageincomeimpknn,weeksworked16imp,parentincomeimp,degltcol,degcol,degadv
personid,,,,,,
100061,"12,500",48,"7,400",1,0,0
100139,"120,000",53,"57,000",1,0,0
100284,"58,000",47,"50,000",1,0,0
100292,"61,639",4,"62,760",0,1,0
100583,"30,000",53,"18,500",1,0,0
100833,"39,000",45,"37,000",1,0,0
100931,"56,000",53,"60,200",0,1,0
101089,"36,000",53,"32,307",1,0,0
101122,"31,685",39,"46,362",1,0,0


In [30]:
# -----------------------
# Inspect and compare data
# -----------------------

# Join selected imputed columns to the original selection for side-by-side viewing.
wagedata_view = (
    wagedata
    .join(wagedataimpknn[['wageincomeimpknn', 'weeksworked16imp', 'parentincomeimp']])
)

# Peek at the first 10 rows of raw vs. imputed values (plus the education dummies).
wagedata_view[['wageincome', 'weeksworked16', 'parentincome', 'degcol', 'degadv', 'wageincomeimpknn']].head(10)



,wageincome,weeksworked16,parentincome,degcol,degadv,wageincomeimpknn
personid,,,,,,
100061,"12,500",48,"7,400",0,0,"12,500"
100139,"120,000",53,"57,000",0,0,"120,000"
100284,"58,000",47,"50,000",0,0,"58,000"
100292,NaN,4,"62,760",1,0,"61,639"
100583,"30,000",53,"18,500",0,0,"30,000"
100833,"39,000",45,"37,000",0,0,"39,000"
100931,"56,000",53,"60,200",1,0,"56,000"
101089,"36,000",53,"32,307",0,0,"36,000"
101122,NaN,39,"46,362",0,0,"31,685"


In [31]:
# Simple summary statistics to compare original vs. imputed wage income.
wage_summary = wagedata_view[['wageincome', 'wageincomeimpknn']].agg(['count', 'mean', 'std'])
print(wage_summary)



       wageincome  wageincomeimpknn
count       5,091             8,984
mean       49,477            46,883
std        40,678            32,076


In [32]:
nls97 = nls97.join(wagedataimpknn[['wageincomeimpknn', 'weeksworked16imp', 'parentincomeimp']])
nls97[['wageincome', 'wageincomeimpknn','weeksworked16', 'weeksworked16imp', 'parentincome', 'parentincomeimp']].head(10)

,wageincome,wageincomeimpknn,weeksworked16,weeksworked16imp,parentincome,parentincomeimp
personid,,,,,,
100061,"12,500","12,500",48,48,"7,400","7,400"
100139,"120,000","120,000",53,53,"57,000","57,000"
100284,"58,000","58,000",47,47,"50,000","50,000"
100292,NaN,"61,639",4,4,"62,760","62,760"
100583,"30,000","30,000",53,53,"18,500","18,500"
100833,"39,000","39,000",45,45,"37,000","37,000"
100931,"56,000","56,000",53,53,"60,200","60,200"
101089,"36,000","36,000",53,53,"32,307","32,307"
101122,NaN,"31,685",39,39,"46,362","46,362"


KNN does imputations without making any assumptions about the distribution of
the underlying data. With regression imputation, the standard assumptions for linear
regression apply – that is, that there is a linear relationship between features and that
they are distributed normally. If this is not the case, KNN is likely a better approach for
imputation.
Despite these advantages, KNN imputation does have limitations. First, we must tune
the model with an initial assumption about a good value for k, sometimes informed by
little more than our knowledge of the size of the dataset. KNN is also computationally
expensive and may be impractical for very large datasets. Finally, KNN imputation may
not perform well when the correlation is weak between the feature to be imputed and the
predictor features. An alternative to KNN for imputation, random forest imputation, can
help us avoid the disadvantages of both KNN and regression imputation. We will explore
random forest imputation in the next section.

### Further options - Random forests for imputation